In [1]:
#!pip install opencv-python
#!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
#!pip install mediapipe
#!pip install opencv-python transformers accelerate


In [2]:
import time
t = str(int(time.time()))+"_"
t

'1728759673_'

In [3]:
from diffusers import AutoencoderKL, StableDiffusionXLControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import os

from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler
from controlnet_aux import MidasDetector

from PIL import Image
import torch
import random
import numpy as np
import cv2

2024-10-12 19:01:19.452738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 19:01:19.452821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 19:01:19.452856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
controlnet_conditioning_scale = 1.0  
prompt = "A detailed, high-quality photograph, high-quality  realistically, well lit, of a human palm on a pure white background. White background.The palm is facing the viewer, fingers extended, with the pattern of creases and lines on the hand clearly visible. There are no other objects or elements in the image."
negative_prompt = "not a palm, drawn, cartoons, animation, drawing, Blurriness, (many folds), low resolution, other body parts, extraneous objects, background elements, jewelry, tattoos, text, watermarks, multiple hands, gloves, nail polish, distracting details."



controlnet = ControlNetModel.from_pretrained(
    "xinsir/controlnet-depth-sdxl-1.0",
    torch_dtype=torch.float16, cache_dir='./modele cache/'
)

# when test with other base model, you need to change the vae also.
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, cache_dir='./modele cache/')

eulera_scheduler = EulerAncestralDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler",cache_dir='./modele cache/')


pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "John6666/realism-by-stable-yogi-sdxlv2-sdxl",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
    scheduler=eulera_scheduler,
    cache_dir='./modele cache/'
).to("cuda")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
%%time
import torch
import os
import random
from PIL import Image
import numpy as np
import mediapipe as mp


# Инициализация MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=True,        # Режим обработки статичного изображения
    max_num_hands=1,               # Максимальное количество рук для обнаружения
    min_detection_confidence=0.95   # Установите порог для обнаружения
)



for i in range(200):
    # Получаем случайный файл маски
    mask_files = [f for f in os.listdir('./masks/') if f.endswith((".jpg", ".png"))]
    random_mask_file = random.choice(mask_files)
    controlnet_img = Image.open('./masks/' + random_mask_file).convert("L")  # Преобразуем в черно-белый формат
    width, height = controlnet_img.size

    # Генерация изображения с pipe
    images = pipe(
        prompt,
        negative_prompt=negative_prompt,
        image=controlnet_img,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        width=width,
        height=height,
        num_inference_steps=30,
        num_images_per_prompt=10
    )

    # Конвертируем маску в NumPy массив для применения
    controlnet_mask = np.array(controlnet_img)

    # Пройдем по всем сгенерированным изображениям
    for im in range(len(images.images)):
        #images.images[im].save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
        ###############################
        # Получаем сгенерированное изображение как NumPy массив
        img_array = np.array(images.images[im])
        # Создаем белый фон, такого же размера как изображение
        white_background = np.full_like(img_array, 255)
        # Применяем маску: где маска черная (0), делаем белым
        masked_image = np.where(controlnet_mask[..., None] <100, white_background, img_array)
        # Конвертируем обратно в PIL изображение
        masked_pil_image = Image.fromarray(masked_image)
        # Сохраняем изображение
        #masked_pil_image.save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
        #############################

        # Преобразование PIL Image в NumPy массив с цветовым пространством RGB
        image_np = np.array(masked_pil_image.convert('RGB'))
        
        # Обработка изображения
        results = hands.process(image_np)
        
        # Проверка и получение оценки уверенности
        if results.multi_hand_landmarks:
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                # Получение информации о классификации руки
                hand_classification = results.multi_handedness[idx]
                hand_label = hand_classification.classification[0].label  # 'Left' или 'Right'
                hand_score = hand_classification.classification[0].score  # Оценка уверенности (от 0 до 1)
                probability = hand_score * 100  # Конвертируем в проценты
        
                # Вывод информации
                print(f"вероятность {probability:.2f}%")

                masked_pil_image.save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i)+f"_{probability:.2f}" + ".jpg")

        else:
            #masked_pil_image.save("./img_b/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
            print("Рука не обнаружена на изображении.")
    
    # Очищаем кэш после каждой итерации
    torch.cuda.empty_cache()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1728759696.498418   37937 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728759696.539393   37937 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


  0%|          | 0/30 [00:00<?, ?it/s]

W0000 00:00:1728759742.885902   37970 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/root/miniconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


вероятность 97.93%
Рука не обнаружена на изображении.
Рука не обнаружена на изображении.
Рука не обнаружена на изображении.
вероятность 97.77%
вероятность 98.00%
вероятность 95.31%
вероятность 98.33%
вероятность 96.76%
Рука не обнаружена на изображении.


  0%|          | 0/30 [00:00<?, ?it/s]

вероятность 96.72%
вероятность 96.75%
вероятность 96.39%
Рука не обнаружена на изображении.
вероятность 97.62%
вероятность 96.16%
вероятность 96.97%
вероятность 95.98%
вероятность 85.86%
Рука не обнаружена на изображении.


  0%|          | 0/30 [00:00<?, ?it/s]

вероятность 91.33%
вероятность 97.59%
вероятность 93.37%
вероятность 97.77%
вероятность 98.52%
вероятность 98.02%
Рука не обнаружена на изображении.
вероятность 97.38%
вероятность 97.19%
вероятность 94.48%


  0%|          | 0/30 [00:00<?, ?it/s]


KeyboardInterrupt

